####Loading the Data from Kaggle

In [1]:
import pandas as pd
import pickle
import numpy as np
import tensorflow as tf

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tobiasbueck/multilingual-customer-support-tickets")

print("Path to dataset files:", path)

100%|██████████| 16.1M/16.1M [00:00<00:00, 107MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/tobiasbueck/multilingual-customer-support-tickets/versions/14


In [3]:
import os

path_to_dataset = "/root/.cache/kagglehub/datasets/tobiasbueck/multilingual-customer-support-tickets/versions/14"

os.listdir(path_to_dataset)

['dataset-tickets-german_normalized.csv',
 'dataset-tickets-german_normalized_50_5_2.csv',
 'aa_dataset-tickets-multi-lang-5-2-50-version.csv',
 'dataset-tickets-multi-lang-4-20k.csv',
 'dataset-tickets-multi-lang3-4k.csv']

In [6]:
BASE_DIR = "/root/.cache/kagglehub/datasets/tobiasbueck/multilingual-customer-support-tickets/versions/14/dataset-tickets-multi-lang-4-20k.csv"

df = pd.read_csv(BASE_DIR)

print("Dataset has ", df.shape[0], " rows and ", df.shape[1], " columns")
print("Columns", list(df.columns))

Dataset has  20000  rows and  15  columns
Columns ['subject', 'body', 'answer', 'type', 'queue', 'priority', 'language', 'tag_1', 'tag_2', 'tag_3', 'tag_4', 'tag_5', 'tag_6', 'tag_7', 'tag_8']


####Analyzing the dataset

In [7]:
text_columns = []

if "subject" in df.columns and "body" in df.columns:
  text_columns = ["subject", "body"]
  print("Text columns: subject, body")
elif "subject" in df.columns:
  text_columns = ["subject"]
  print("Text columns: subject")
else:
  other_text = ["description", "text", "content"]

  for col in other_text:
    if col in df.columns:
     text_columns = [col]
     print("Text columns: ", text_columns)
     break


Text columns: subject, body


In [8]:
label_col = None
labels = ["queue", "tag_1", "label", "category", "department"]

for col in labels:
  if col in df.columns:
    label_col = col
    print("Label column: ", label_col)
    break

if label_col is None:
  print("No label column found")

Label column:  queue


####Creating model input

In [9]:
if "subject" in df.columns:
  df["subject"] = df["subject"].fillna("")

if "body" in df.columns:
  df["body"] = df["body"].fillna("")

In [11]:
if "subject" in df.columns and "body" in df.columns:
  df["Final_Text"] = df["subject"] + " " + df["body"]
else:
  df["Final_Text"] = df[text_columns[0]]

In [12]:
df["Final_Text"] = df["Final_Text"].str.strip()

print("Sample Final text: ", df["Final_Text"][0])

Sample Final text:  Unvorhergesehener Absturz der Datenanalyse-Plattform Die Datenanalyse-Plattform brach unerwartet ab, da die Speicheroberfläche zu gering war. Ich habe versucht, Laravel 8 und meinen MacBook Pro neu zu starten, aber das Problem behält sich bei. Ich benötige Ihre Unterstützung, um diesen Fehler zu beheben.


####Defining model output

In [14]:
queue_count = df[label_col].value_counts()
print("unique queues:", len(queue_count))

unique queues: 10


In [17]:
print(queue_count.head(10))

queue
Technical Support                  5824
Product Support                    3708
Customer Service                   3152
IT Support                         2292
Billing and Payments               2086
Returns and Exchanges              1001
Service Outages and Maintenance     764
Sales and Pre-Sales                 572
Human Resources                     338
General Inquiry                     263
Name: count, dtype: int64
